In [ ]:
%%capture
pip install psycopg2-binary boto3

In [ ]:
import os
import boto3
import json
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
def get_secret(secret_name):
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    get_secret_value_response = json.loads(get_secret_value_response['SecretString'])
    return get_secret_value_response

In [ ]:
db_credentials = get_secret(secret_name='dev/detraining/rds')
db_credentials

In [ ]:
USERNAME = db_credentials["sparsh_rds_postgres_username"]
PASSWORD = db_credentials["sparsh_rds_postgres_password"]
HOST = db_credentials["sparsh_rds_postgres_host"]
PORT = db_credentials["sparsh_rds_postgres_port"]

DBNAME1 = "postgres"
DBNAME2 = "mydb"

CONN1 = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DBNAME1}"
CONN2 = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DBNAME2}"

In [ ]:
engine = create_engine(CONN1)

In [ ]:
engine.execute('SELECT datname FROM pg_database;').fetchall()

[('template0',),
 ('template1',),
 ('postgres',),
 ('rdsadmin',),
 ('mydb',),
 ('studentdb',)]

In [ ]:
conn = engine.connect()

try: 
    conn.execution_options(isolation_level="AUTOCOMMIT").execute("CREATE DATABASE mydb")
except Exception as e:
    print(e)
finally:
    conn.close()

In [ ]:
engine.execute('SELECT datname FROM pg_database;').fetchall()

[('template0',),
 ('template1',),
 ('postgres',),
 ('rdsadmin',),
 ('mydb',),
 ('studentdb',)]

Create Table for students which includes below columns

- student_id <br>
- name <br>
- age <br>
- gender <br>
- subject <br>
- marks

In [ ]:
engine = create_engine(CONN2)

In [ ]:
conn = engine.connect()

try: 
    conn.execute("CREATE TABLE IF NOT EXISTS students (student_id int, name varchar,\
    age int, gender varchar, subject varchar, marks int);")
except Exception as e: 
    print("Error: Issue creating table")
    print(e)
finally:
    conn.close()

Insert the following two rows in the table:
- First Row:  1, "Raj", 23, "Male", "Python", 85
- Second Row: 2, "Priya", 22, "Female", "Python", 86

In [ ]:
conn = engine.connect()

try: 
    conn.execute("INSERT INTO students (student_id, name, age, gender, subject, marks) \
                 VALUES (%s, %s, %s, %s, %s, %s)", \
                 (1, "Raj", 23, "Male", "Python", 85))
    conn.execute("INSERT INTO students (student_id, name, age, gender, subject, marks) \
                  VALUES (%s, %s, %s, %s, %s, %s)",
                  ( 2, "Priya", 22, "Female", "Python", 86))
except Exception as e: 
    print("Error: Inserting Rows")
    print (e)
finally:
    conn.close()

In [ ]:
conn = engine.connect()

try: 
    result = conn.execute("SELECT * FROM students;")
except Exception as e: 
    print("Error: select *")
    print(e)

row = result.fetchone()
while row:
    print(row)
    row = result.fetchone()

(1, 'Raj', 23, 'Male', 'Python', 85)
(2, 'Priya', 22, 'Female', 'Python', 86)


In [ ]:
conn = engine.connect()
pd.read_sql("SELECT * FROM students", con=conn)

,student_id,name,age,gender,subject,marks
0,1,Raj,23,Male,Python,85
1,2,Priya,22,Female,Python,86


In [ ]:
!wget -q --show-progress https://github.com/RecoHut-Datasets/retail_general/raw/main/sample_superstore.csv

sample_superstore.c 100%[===================>]   2.01M  --.-KB/s    in 0.06s   


In [ ]:
!head sample_superstore.csv

category,city,country,customer_name,discount,number_of_records,order_date,order_id,postal_code,manufacturer,product_name,profit,quantity,region,sales,segment,ship_date,ship_mode,state,sub_category
Furniture,Henderson,United States,Claire Gute,0,1,2017-11-08,CA-2017-152156,42420,Bush,Bush Somerset Collection Bookcase,41.9136,2,South,261.96,Consumer,2017-11-11,Second Class,Kentucky,Bookcases
Furniture,Henderson,United States,Claire Gute,0,1,2017-11-08,CA-2017-152156,42420,Hon,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",219.582,3,South,731.94,Consumer,2017-11-11,Second Class,Kentucky,Chairs
Office Supplies,Los Angeles,United States,Darrin Van Huff,0,1,2017-06-12,CA-2017-138688,90036,Universal,Self-Adhesive Address Labels for Typewriters by Universal,6.8714,2,West,14.62,Corporate,2017-06-16,Second Class,California,Labels
Furniture,Fort Lauderdale,United States,Sean O'Donnell,0.45,1,2016-10-11,US-2016-108966,33311,Bretford,Bretford CR4500 Series Slim Rectangular Table,-383

In [ ]:
data = pd.read_csv('sample_superstore.csv')
data

,category,city,country,customer_name,discount,number_of_records,order_date,order_id,postal_code,manufacturer,product_name,profit,quantity,region,sales,segment,ship_date,ship_mode,state,sub_category
0,Furniture,Henderson,United States,Claire Gute,0.00,1,2017-11-08,CA-2017-152156,42420.0,Bush,Bush Somerset Collection Bookcase,41.9136,2,South,261.9600,Consumer,2017-11-11,Second Class,Kentucky,Bookcases
1,Furniture,Henderson,United States,Claire Gute,0.00,1,2017-11-08,CA-2017-152156,42420.0,Hon,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",219.5820,3,South,731.9400,Consumer,2017-11-11,Second Class,Kentucky,Chairs
2,Office Supplies,Los Angeles,United States,Darrin Van Huff,0.00,1,2017-06-12,CA-2017-138688,90036.0,Universal,Self-Adhesive Address Labels for Typewriters b...,6.8714,2,West,14.6200,Corporate,2017-06-16,Second Class,California,Labels
3,Furniture,Fort Lauderdale,United States,Sean O'Donnell,0.45,1,2016-10-11,US-2016-108966,33311.0,Bretford,Bretford CR4500 Series Slim Rectangular Table,-383.0310,5,South,957.5775,Consumer,2016-10-18,Standard Class,Florida,Tables
4,Office Supplies,Fort Lauderdale,United States,Sean O'Donnell,0.20,1,2016-10-11,US-2016-108966,33311.0,Eldon,Eldon Fold 'N Roll Cart System,2.5164,2,South,22.3680,Consumer,2016-10-18,Standard Class,Florida,Storage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,Technology,Los Angeles,United States,Ben Ferrer,0.20,1,2015-12-24,US-2015-138247,90045.0,Other,Seidio BD2-HK3IPH5-BK DILEX Case and Holster C...,17.3656,7,West,173.6560,Home Office,2015-12-29,Standard Class,California,Phones
9990,Technology,Los Angeles,United States,Ben Ferrer,0.20,1,2015-07-12,CA-2015-110184,90036.0,GE,GE DSL Phone Line Filter,28.7928,8,West,255.9360,Home Office,2015-07-16,Standard Class,California,Phones
9991,Technology,Santa Barbara,United States,Maria Etezadi,0.20,1,2017-06-06,CA-2017-160717,93101.0,Samsung,Samsung Galaxy Mega 6.3,226.7946,9,West,3023.9280,Home Office,2017-06-11,Standard Class,California,Phones
9992,Technology,Yuma,United States,Gary Hansen,0.70,1,2018-10-16,CA-2018-159282,85364.0,Other,Swingline SM12-08 MicroCut Jam Free Shredder,-479.9880,5,West,599.9850,Home Office,2018-10-21,Standard Class,Arizona,Machines


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   category           9994 non-null   object 
 1   city               9994 non-null   object 
 2   country            9994 non-null   object 
 3   customer_name      9994 non-null   object 
 4   discount           9994 non-null   float64
 5   number_of_records  9994 non-null   int64  
 6   order_date         9994 non-null   object 
 7   order_id           9994 non-null   object 
 8   postal_code        9983 non-null   float64
 9   manufacturer       9994 non-null   object 
 10  product_name       9994 non-null   object 
 11  profit             9994 non-null   float64
 12  quantity           9994 non-null   int64  
 13  region             9994 non-null   object 
 14  sales              9994 non-null   float64
 15  segment            9994 non-null   object 
 16  ship_date          9994 

In [ ]:
data.to_sql('superstore',
            con=engine,
            if_exists='replace',
            index=False,
            method='multi')

In [ ]:
conn = engine.connect()
pd.read_sql("SELECT * FROM superstore LIMIT 10", con=conn)

,category,city,country,customer_name,discount,number_of_records,order_date,order_id,postal_code,manufacturer,product_name,profit,quantity,region,sales,segment,ship_date,ship_mode,state,sub_category
0,Furniture,Henderson,United States,Claire Gute,0.00,1,2017-11-08,CA-2017-152156,42420.0,Bush,Bush Somerset Collection Bookcase,41.9136,2,South,261.9600,Consumer,2017-11-11,Second Class,Kentucky,Bookcases
1,Furniture,Henderson,United States,Claire Gute,0.00,1,2017-11-08,CA-2017-152156,42420.0,Hon,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",219.5820,3,South,731.9400,Consumer,2017-11-11,Second Class,Kentucky,Chairs
2,Office Supplies,Los Angeles,United States,Darrin Van Huff,0.00,1,2017-06-12,CA-2017-138688,90036.0,Universal,Self-Adhesive Address Labels for Typewriters b...,6.8714,2,West,14.6200,Corporate,2017-06-16,Second Class,California,Labels
3,Furniture,Fort Lauderdale,United States,Sean O'Donnell,0.45,1,2016-10-11,US-2016-108966,33311.0,Bretford,Bretford CR4500 Series Slim Rectangular Table,-383.0310,5,South,957.5775,Consumer,2016-10-18,Standard Class,Florida,Tables
4,Office Supplies,Fort Lauderdale,United States,Sean O'Donnell,0.20,1,2016-10-11,US-2016-108966,33311.0,Eldon,Eldon Fold 'N Roll Cart System,2.5164,2,South,22.3680,Consumer,2016-10-18,Standard Class,Florida,Storage
5,Furniture,Los Angeles,United States,Brosina Hoffman,0.00,1,2015-06-09,CA-2015-115812,90032.0,Eldon,Eldon Expressions Wood and Plastic Desk Access...,14.1694,7,West,48.8600,Consumer,2015-06-14,Standard Class,California,Furnishings
6,Office Supplies,Los Angeles,United States,Brosina Hoffman,0.00,1,2015-06-09,CA-2015-115812,90032.0,Newell,Newell 322,1.9656,4,West,7.2800,Consumer,2015-06-14,Standard Class,California,Art
7,Technology,Los Angeles,United States,Brosina Hoffman,0.20,1,2015-06-09,CA-2015-115812,90032.0,Mitel,Mitel 5320 IP Phone VoIP phone,90.7152,6,West,907.1520,Consumer,2015-06-14,Standard Class,California,Phones
8,Office Supplies,Los Angeles,United States,Brosina Hoffman,0.20,1,2015-06-09,CA-2015-115812,90032.0,DXL,DXL Angle-View Binders with Locking Rings by S...,5.7825,3,West,18.5040,Consumer,2015-06-14,Standard Class,California,Binders
9,Office Supplies,Los Angeles,United States,Brosina Hoffman,0.00,1,2015-06-09,CA-2015-115812,90032.0,Belkin,Belkin F5C206VTEL 6 Outlet Surge,34.4700,5,West,114.9000,Consumer,2015-06-14,Standard Class,California,Appliances


---

In [ ]:
query = """
CREATE TABLE Music (
    Artist VARCHAR(20) NOT NULL, 
    SongTitle VARCHAR(30) NOT NULL,
    AlbumTitle VARCHAR(25),
    Year INT,
    Price FLOAT,
    Genre VARCHAR(10),
    CriticRating FLOAT,
    Tags TEXT,
    PRIMARY KEY(Artist, SongTitle)
);
"""

conn = engine.connect()

try: 
    result = conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Year, Price, Genre, CriticRating, 
    Tags)
VALUES(
    'No One You Know', 'Call Me Today', 'Somewhat Famous',
    2015, 2.14, 'Country', 7.8,
    '{"Composers": ["Smith", "Jones", "Davis"],"LengthInSeconds": 214}'
);
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Price, Genre, CriticRating)
VALUES(
    'No One You Know', 'My Dog Spot', 'Hey Now',
    1.98, 'Country', 8.4
);
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Price, Genre)
VALUES(
    'The Acme Band', 'Look Out, World', 'The Buck Starts Here',
    0.99, 'Rock'
);
INSERT INTO Music 
    (Artist, SongTitle, AlbumTitle, 
    Price, Genre, 
    Tags)
VALUES(
    'The Acme Band', 'Still In Love', 'The Buck Starts Here',
    2.47, 'Rock', 
    '{"radioStationsPlaying": ["KHCR", "KBQX", "WTNR", "WJJH"], "tourDates": { "Seattle": "20150625", "Cleveland": "20150630"}, "rotation": Heavy}'
);
"""

conn = engine.connect()

try: 
    result = conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
SELECT * FROM Music;
"""

conn = engine.connect()
pd.read_sql(query, conn)

,artist,songtitle,albumtitle,year,price,genre,criticrating,tags
0,No One You Know,Call Me Today,Somewhat Famous,2015.0,2.14,Country,7.8,"{""Composers"": [""Smith"", ""Jones"", ""Davis""],""Len..."
1,No One You Know,My Dog Spot,Hey Now,NaN,1.98,Country,8.4,None
2,The Acme Band,"Look Out, World",The Buck Starts Here,NaN,0.99,Rock,NaN,None
3,The Acme Band,Still In Love,The Buck Starts Here,NaN,2.47,Rock,NaN,"{""radioStationsPlaying"": [""KHCR"", ""KBQX"", ""WTN..."


In [ ]:
query = """
SELECT * FROM Music
WHERE Artist='No One You Know';
"""

conn = engine.connect()
pd.read_sql(query, conn)

,artist,songtitle,albumtitle,year,price,genre,criticrating,tags
0,No One You Know,Call Me Today,Somewhat Famous,2015.0,2.14,Country,7.8,"{""Composers"": [""Smith"", ""Jones"", ""Davis""],""Len..."
1,No One You Know,My Dog Spot,Hey Now,NaN,1.98,Country,8.4,None


In [ ]:
query = """
SELECT * FROM Music
WHERE Artist='No One You Know' AND SongTitle LIKE 'Call%%';
"""

conn = engine.connect()
pd.read_sql(query, conn)

,artist,songtitle,albumtitle,year,price,genre,criticrating,tags
0,No One You Know,Call Me Today,Somewhat Famous,2015,2.14,Country,7.8,"{""Composers"": [""Smith"", ""Jones"", ""Davis""],""Len..."


In [ ]:
query = """
SELECT * FROM Music
WHERE Artist='No One You Know' AND SongTitle LIKE '%%Today%%'
AND Price > 1.00;
"""

conn = engine.connect()
pd.read_sql(query, conn)

,artist,songtitle,albumtitle,year,price,genre,criticrating,tags
0,No One You Know,Call Me Today,Somewhat Famous,2015,2.14,Country,7.8,"{""Composers"": [""Smith"", ""Jones"", ""Davis""],""Len..."


In [ ]:
query = """
UPDATE Music
SET Genre = 'Disco'
WHERE Artist = 'The Acme Band' AND SongTitle = 'Still In Love';
"""

conn = engine.connect()

try: 
    conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
SELECT * FROM Music;
"""

conn = engine.connect()
pd.read_sql(query, conn)

,artist,songtitle,albumtitle,year,price,genre,criticrating,tags
0,No One You Know,Call Me Today,Somewhat Famous,2015.0,2.14,Country,7.8,"{""Composers"": [""Smith"", ""Jones"", ""Davis""],""Len..."
1,No One You Know,My Dog Spot,Hey Now,NaN,1.98,Country,8.4,None
2,The Acme Band,"Look Out, World",The Buck Starts Here,NaN,0.99,Rock,NaN,None
3,The Acme Band,Still In Love,The Buck Starts Here,NaN,2.47,Disco,NaN,"{""radioStationsPlaying"": [""KHCR"", ""KBQX"", ""WTN..."


In [ ]:
query = """
DELETE FROM Music
WHERE Artist = 'The Acme Band' AND SongTitle = 'Look Out, World';
"""

conn = engine.connect()

try: 
    conn.execute(query)
except Exception as e: 
    print(e)

In [ ]:
query = """
SELECT * FROM Music;
"""

conn = engine.connect()
pd.read_sql(query, conn)

,artist,songtitle,albumtitle,year,price,genre,criticrating,tags
0,No One You Know,Call Me Today,Somewhat Famous,2015.0,2.14,Country,7.8,"{""Composers"": [""Smith"", ""Jones"", ""Davis""],""Len..."
1,No One You Know,My Dog Spot,Hey Now,NaN,1.98,Country,8.4,None
2,The Acme Band,Still In Love,The Buck Starts Here,NaN,2.47,Disco,NaN,"{""radioStationsPlaying"": [""KHCR"", ""KBQX"", ""WTN..."


In [ ]:
query = """
DROP TABLE Music;
"""

conn = engine.connect()

try: 
    conn.execute(query)
except Exception as e: 
    print(e)

---